In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 34 kB/s 
     |████████████████████████████████| 198 kB 44.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=4cffc0e59db3a175945e646a879ae71b8c5a02b454154bbe96429dd4c71a78e6
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('spotify_classification')\
        .getOrCreate()

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = spark.read.json("drive/My Drive/spot_data.json")

In [5]:
from pyspark.ml.feature import Bucketizer
bucketBorders=[0,10,20,30,40,50,60,70,80,90,100]

In [6]:
bucketer=Bucketizer().setSplits(bucketBorders).setInputCol("popularity_track").setOutputCol("label")
df_buck = bucketer.transform(df)

In [15]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [20]:
assembler = VectorAssembler( 
inputCols=['age',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df_buck)

In [21]:
final_data = output.select( "features", "label")

In [22]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [18]:
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

In [19]:
ovr = OneVsRest(classifier=lr)

In [23]:
ovrModel = ovr.fit(train)

In [24]:
predictions = ovrModel.transform(test)

In [25]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [26]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.659921
